In [1]:
import json
from watson_developer_cloud import ToneAnalyzerV3

## Ask for credentials

In [2]:
# not the best approach here but it will do
import getpass
print("conversation username:")
username = getpass.getpass()

print("conversation password:")
password = getpass.getpass()

conversation username:
········
conversation password:
········


## Create a ToneAnalyzer connector object

In [3]:
tone_analyzer = ToneAnalyzerV3(
   username=username,
   password=password,
   version='2016-05-19')

## Function for parsing ToneAnalyzer JSON response

In [4]:
def parse_toneanalyzer_response(json_data):
    """Parses the JSON response from ToneAnalyzer to return
    a dictionary of emotions and their corresponding score.

    Parameters
    ----------
    json_data: {dict} a json response from ToneAnalyzer (see Notes)

    Returns
    -------
    dict : a {dict} whose keys are emotion ids and values are their corresponding score.

    Notes
    -----
    for an example of json see type pytest_data/tones_1.json
    """
    emotions = {}
    for entry in json_data['document_tone']['tone_categories']:
        if entry['category_id'] == 'emotion_tone':
            for emotion in entry['tones']:
                emotion_key = emotion['tone_name']
                emotion_value = emotion['score']
                emotions[emotion_key] = emotion_value
    return(emotions)

In [5]:
json_response = tone_analyzer.tone(text="I'm so pumped up by rocking that whole demo!")
print(parse_toneanalyzer_response(json_response))

{u'Anger': 0.118448, u'Joy': 0.666785, u'Fear': 0.044258, u'Sadness': 0.099872, u'Disgust': 0.090883}


## Load `textbooks.csv` into a pandas dataframe

In [6]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_ed0baafd744e4452b0ef8e582e0f83a3(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_536659497418808f634cd60a91b6c12706d9d46a','domain': {'id': 'c00ac61e4791413396fb2d1701473203'},
            'password': 'u1a4)e~Do#-PEyj#'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_ed0baafd744e4452b0ef8e582e0f83a3('WatsonTest', 'textbook_reviews_formatted.csv'))
df_data_1.head()

,Unnamed: 0,asin,reviewText,title,price
0,0,0070428077,"There is no real formatting in this book, just...",Machine Learning,50.27
1,1,0070428077,I read this book about 7 years ago while in th...,Machine Learning,50.27
2,2,0070428077,If you are interested in the topic of machine ...,Machine Learning,50.27
3,3,0070428077,This book serves as an excellent introduction ...,Machine Learning,50.27
4,4,0070428077,"Intelligent machines can be characterized, at ...",Machine Learning,50.27


## Process all reviews via ToneAnalyzer

In [7]:
data_map = {}

for index,row in df_data_1.iterrows():
    #print(row['reviewText'])
    response = tone_analyzer.tone(text=row['reviewText'])
    data_map[row[0]] = parse_toneanalyzer_response(response)
    #print(data_map[row[0]])

In [9]:
df_data_1['joy'] = df_data_1['Unnamed: 0'].map(lambda x : data_map[x].get('Joy',0.0))
df_data_1['anger'] = df_data_1['Unnamed: 0'].map(lambda x : data_map[x].get('Anger',0.0))
df_data_1['fear'] = df_data_1['Unnamed: 0'].map(lambda x : data_map[x].get('Fear',0.0))
df_data_1['sadness'] = df_data_1['Unnamed: 0'].map(lambda x : data_map[x].get('Sadness',0.0))
df_data_1['disgust'] = df_data_1['Unnamed: 0'].map(lambda x : data_map[x].get('Disgust',0.0))

In [10]:
df_data_1.groupby('asin').mean().sort_values(by='joy', ascending=False)

,Unnamed: 0,price,joy,anger,fear,sadness,disgust
asin,,,,,,,
0262193981,89.0,58.49,0.602187,0.103348,0.050392,0.339367,0.038366
1461471370,267.0,56.47,0.592355,0.075750,0.041199,0.184450,0.040237
1617290181,296.0,29.48,0.580712,0.118559,0.062267,0.266279,0.031640
1782161406,309.5,18.49,0.558162,0.108174,0.094054,0.437589,0.061358
0321321367,100.0,59.07,0.556551,0.133806,0.186068,0.294485,0.101630
0387952845,136.0,18.60,0.532165,0.109921,0.081381,0.234393,0.053619
1449367615,257.5,31.20,0.516575,0.138582,0.054572,0.255586,0.038545
1420067184,199.5,40.50,0.514658,0.123667,0.084937,0.456737,0.066764
0387310738,114.0,25.60,0.508823,0.143709,0.107897,0.336323,0.057385
